In [62]:
import numpy as np
from numpy import *
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import random
from numpy.linalg import eig
import copy
from copy import deepcopy
import math
import os
import sys
import operator 
from scipy.special import logsumexp
eps = np.finfo(float).eps

In [63]:
def train_test_split(df, test_size):
    if isinstance(test_size, float):
        test_size = round(test_size * len(df))
    indices = df.index.tolist()
    test_indices = random.sample(population=indices, k=test_size)
    
    test_df = df.loc[test_indices]
    train_df = df.drop(test_indices)
    return train_df, test_df

In [64]:
def sigmoid(gamma):
    if gamma < 0:
        return 1 - 1/(1 + math.exp(gamma))
    else:
        return 1/(1 + math.exp(-gamma))

In [65]:
def initialization():
    weights1 = np.zeros((N, 784))
    weights2 = np.random.random((10,N))
    bias1 = [0 for i in range(N)]
    bias2 = [random.random() for i in range(10)]
    a1 = [0 for i in range(N)]
    h = [0 for i in range(N)]
    a2 = [0 for i in range(10)]
    y = [0.12 for i in range(10)]
    return weights1, weights2, bias1, bias2, a1, h, a2, y

In [77]:
def hiddenLayer(trainData, weights1, a1, bias1, h):
    for i in range(N):
        for j in range(784):
            a1[i] += weights1[i][j] * trainData[j]
        a1[i] += bias1[i]
        h[i] = sigmoid(a1[i])
    return a1, h

In [67]:
def outputLayer(trainData, weights2, h, a2, bias2):
    for i in range(10):
        for j in range(N):
            a2[i] += weights2[i][j] * h[j]
        a2[i] += bias2[i]
    return a2

In [68]:
def softmax(a2):
    denom, b = 0, []
    denom = max(a2)
    denom = np.exp(denom)
    for i in range(10):
        b.append(np.exp(a2[i]) / denom)
    return b

In [69]:
def update2(y, output, weights2, h, bias2):
    delJa2, eta = [], 0.5
    for i in range(10):
        if(i == output):
            delJa2.append(y[i]-1)
        else:
            delJa2.append(y[i])
    for i in range(10):
        for j in range(N):
            weights2[i][j] -= eta * delJa2[i] * h[j]
        bias2[i] -= eta * delJa2[i]
    return weights2, bias2, delJa2

In [70]:
def update1(trainData, weights1, weights2, bias1, a1, delJa2):
    delJa1, eta = [], 0.5
    for i in range(N):
        for j in range(10):
            delJa1.append(delJa2[j] * weights2[j][i] * sigmoid(a1[i]) * (1 - sigmoid(a1[i])))
    for i in range(N):
        for j in range(784):
            weights1[i][j] -= eta * delJa1[i] * trainData[j]
        bias1[i] -= eta * delJa1[i]
    return weights1, bias1

In [71]:
def neuralNetwork(trainData, train, target):
    weights1, weights2, bias1, bias2, a1, h, a2, y = initialization()
    output = None
    for row in range(len(trainData)):
        a1, h = hiddenLayer(trainData.iloc[[row]].values[0], weights1, a1, bias1, h)
        a2 = outputLayer(trainData.iloc[[row]].values[0], weights2, h, a2, bias2)
        denom, b = 0, []
        for i in range(10):
            z=[]
            for j in range(10):
                z.append(0)
            z[i]=a2[i]
            b.append(np.exp(logsumexp(z) - logsumexp(a2)))
        output = train[target].iloc[[row]].values[0]
        weights2, bias2, delJa2 = update2(b, output, weights2, h, bias2)
        weights1, bias1 = update1(trainData.iloc[[row]].values[0], weights1, weights2, bias1, a1, delJa2)
    return weights1, weights2, bias1, bias2, b

In [72]:
def dataNormalization(train, test, target):
    trainFeatures = train.drop([target], axis = 1)
    testFeatures = test.drop([target], axis = 1)
    
    trainMean = trainFeatures.mean()
    trainStd = trainFeatures.std()
    trainFeatures = (trainFeatures - trainMean) / trainStd
    testFeatures = (testFeatures - trainMean) / trainStd
    return trainFeatures, testFeatures

In [73]:
if __name__ == "__main__":
    df = pd.read_csv("../input_data/Apparel/apparel-trainval.csv")
    N = 120

In [74]:
target = "label"
train, test = train_test_split(df, test_size = 0.2)
trainData, testData = dataNormalization(train, test, target)
uniqueValues = np.unique(train[target])
uniqueValues = np.unique(test[target])

In [94]:
def testNetwork(weights1,weights2,bias1,bias2,testData, targetdata):
    a1 = [0 for i in range(N)]
    h = [0 for i in range(N)]
    a2 = [0 for i in range(10)]
    our_output = []
    for row in range(len(testData)):
        a1, h = hiddenLayer(testData.iloc[[row]].values[0], weights1, a1, bias1, h)
        a2 = outputLayer(testData.iloc[[row]].values[0], weights2, h, a2, bias2)
        denom, b = 0, []
        for i in range(10):
            z=[]
            for j in range(10):
                z.append(0)
            z[i]=a2[i]
            b.append(np.exp(logsumexp(z) - logsumexp(a2)))
        output = test[target].iloc[[row]].values[0]
        op = b.index(max(b))
        print(op == targetdata.iloc[[row]].values[0])
        our_output.append(op)
    return our_output

In [78]:
weights1, weights2, bias1, bias2, y = neuralNetwork(trainData, train, target)

In [95]:
predictedOutput = testNetwork(weights1,weights2,bias1,bias2,testData, test[target])

False
False
False
False
False
False
False
False
False
False
False
False
False
True
False
False
False
False
False
False
False
False
True
False
True
True
False
False
False
False
True
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
True
False
False
False
False
False
False
True
False
False
False
False
True
False
False
False
True
True
False
False
False
False
False
False
False
False
True
False
False
False
False
False
False
False
False
False
False
False
False
False
True
False
False
False
False
True
False
True
False
False
False
False
True
False
True
False
False
False
False
False
False
False
False
False
False
True
False
False
False
False
False
True
False
False
False
False
True
False
False
False
False
True
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
True
False
False
False
False
False
False
False
False
False
False
True
False
False
False
False
False
False
False
False
True
False
False
False
True
Fals

KeyboardInterrupt: 

In [ ]:
np.unique(predictedOutput)

In [ ]:
bias2

In [ ]:
target = "label"
train, test = train_test_split(df, test_size = 0.8)
uniqueValues = np.unique(train[target])
trainData = train.drop([target], axis = 1)
uniqueValues = np.unique(test[target])
testData = test.drop([target], axis = 1)
unique_counts = train.nunique()
unique_stats = pd.DataFrame(unique_counts).rename(columns = {'index': 'feature', 0: 'nunique'})
unique_stats = unique_stats.sort_values('nunique', ascending = True)
record_single_unique = pd.DataFrame(unique_counts[unique_counts == 1]).reset_index().rename(columns = {'index': 'feature', 
                                                                                                                0: 'nunique'})
ops={}
to_drop = list(record_single_unique['feature'])
    
record_single_unique = record_single_unique
ops['single_unique'] = to_drop
        
print('%d features with a single unique value.\n' % len(ops['single_unique']))